In [42]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [43]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

C:\Users\aleja\AppData\Local\Temp\ipykernel_10960\2990043772.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [44]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

In [45]:
dfs['carePlanGeneral']

,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
0,174826,True,49,Ventilation,Spontaneous - adequate
1,174826,True,49,Care Limitation,Full therapy
2,174826,True,49,Stress Ulcer Prophylaxis,Proton pump inhibitor
3,174826,True,49,Airway,Not intubated/normal airway
4,174826,True,49,DVT Prophylaxis,Compression devices
...,...,...,...,...,...
33143,3334564,True,1,Care Limitation,Full therapy
33144,3334564,True,2231,DVT Prophylaxis,Combined device and drug therapy
33145,3334564,False,1,Ventilation,Spontaneous - adequate
33146,3334564,False,1,Activity,Do not elevate HOB


In [46]:
def clean_respiratoryCare(row, df):
    respiratoryCare = row['currenthistoryseqnum']
           
    return  df[df['patientunitstayid']==row['patientunitstayid']]['currenthistoryseqnum'].max()


def range_currenthistoryseqnum(row, df):
    median = 3.00  # df['currenthistoryseqnum'].median()
    currenthistoryseqnum = row ['currenthistoryseqnum']
    
    if (currenthistoryseqnum < median):
        return 'Low'
    elif currenthistoryseqnum == median:
        return 'Mid'
    else:
        return 'High'  

In [47]:
# Hay 15 filas con valor nulo en la columna cplitemvalue, asi que eliminamos esas filas
dfs['carePlanGeneral'] = dfs['carePlanGeneral'].dropna()

dfs['carePlanGeneral'] = dfs['carePlanGeneral'].drop(columns=[
    'activeupondischarge', 'cplitemoffset', 
], axis=1)

In [50]:
set(dfs['carePlanGeneral']['cplgroup'])

{'Activity',
 'Acuity',
 'Airway',
 'Analgesia',
 'Baseline Status',
 'Calories',
 'Care Limitation',
 'Care Plan Reviewed with Patient/Family',
 'Care Providers',
 'Critical Care Discharge/Transfer Planning',
 'Current Rate',
 'DVT Prophylaxis',
 'Family/Health Care Proxy/Contact Info',
 'Goal Rate',
 'Infectious Disease',
 'Ordered Protocols',
 'Planned Procedures',
 'Pressure Ulcer Treatment',
 'Prognosis',
 'Protein',
 'Psychosocial Status',
 'Route-Status',
 'Safety/Restraints',
 'Sedation',
 'Stress Ulcer Prophylaxis',
 'Ventilation',
 'Volume Status'}

In [48]:
dfs['carePlanGeneral']

,patientunitstayid,cplgroup,cplitemvalue
0,174826,Ventilation,Spontaneous - adequate
1,174826,Care Limitation,Full therapy
2,174826,Stress Ulcer Prophylaxis,Proton pump inhibitor
3,174826,Airway,Not intubated/normal airway
4,174826,DVT Prophylaxis,Compression devices
...,...,...,...
33143,3334564,Care Limitation,Full therapy
33144,3334564,DVT Prophylaxis,Combined device and drug therapy
33145,3334564,Ventilation,Spontaneous - adequate
33146,3334564,Activity,Do not elevate HOB


In [49]:
dfs['carePlanGeneral'].info()
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32134 entries, 0 to 33147
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   patientunitstayid  32134 non-null  int64 
 1   cplgroup           32134 non-null  object
 2   cplitemvalue       32134 non-null  object
dtypes: int64(1), object(2)
memory usage: 1004.2+ KB

